In [1]:
import sqlhelpers as sh

In [2]:
# %load Exercise Files\\Chapters\\sql-chap10.txt
-- 01 update triggers
-- test.db

CREATE TABLE widgetCustomer ( id INTEGER PRIMARY KEY, name TEXT, last_order_id INT );
CREATE TABLE widgetSale ( id INTEGER PRIMARY KEY, item_id INT, customer_id INT, quan INT, price INT );

INSERT INTO widgetCustomer (name) VALUES ('Bob');
INSERT INTO widgetCustomer (name) VALUES ('Sally');
INSERT INTO widgetCustomer (name) VALUES ('Fred');

SELECT * FROM widgetCustomer;

CREATE TRIGGER newWidgetSale AFTER INSERT ON widgetSale
    BEGIN
        UPDATE widgetCustomer SET last_order_id = NEW.id WHERE widgetCustomer.id = NEW.customer_id;
    END
;

INSERT INTO widgetSale (item_id, customer_id, quan, price) VALUES (1, 3, 5, 1995);
INSERT INTO widgetSale (item_id, customer_id, quan, price) VALUES (2, 2, 3, 1495);
INSERT INTO widgetSale (item_id, customer_id, quan, price) VALUES (3, 1, 1, 2995);
SELECT * FROM widgetSale;
SELECT * FROM widgetCustomer;

-- 02 preventing updates
-- test.db

DROP TABLE IF EXISTS widgetSale;

CREATE TABLE widgetSale ( id integer primary key, item_id INT, customer_id INTEGER, quan INT, price INT,
    reconciled INT );
INSERT INTO widgetSale (item_id, customer_id, quan, price, reconciled) VALUES (1, 3, 5, 1995, 0);
INSERT INTO widgetSale (item_id, customer_id, quan, price, reconciled) VALUES (2, 2, 3, 1495, 1);
INSERT INTO widgetSale (item_id, customer_id, quan, price, reconciled) VALUES (3, 1, 1, 2995, 0);
SELECT * FROM widgetSale;

CREATE TRIGGER updateWidgetSale BEFORE UPDATE ON widgetSale
    BEGIN
        SELECT RAISE(ROLLBACK, 'cannot update table "widgetSale"') FROM widgetSale
            WHERE id = NEW.id AND reconciled = 1;
    END
;

BEGIN TRANSACTION;
UPDATE widgetSale SET quan = 9 WHERE id = 2;
END TRANSACTION;

SELECT * FROM widgetSale;

-- 03 timestamps
-- test.db

DROP TABLE IF EXISTS widgetSale;
DROP TABLE IF EXISTS widgetCustomer;

CREATE TABLE widgetCustomer ( id integer primary key, name TEXT, last_order_id INT, stamp TEXT );
CREATE TABLE widgetSale ( id integer primary key, item_id INT, customer_id INTEGER, quan INT, price INT, stamp TEXT );
CREATE TABLE widgetLog ( id integer primary key, stamp TEXT, event TEXT, username TEXT, tablename TEXT, table_id INT);

INSERT INTO widgetCustomer (name) VALUES ('Bob');
INSERT INTO widgetCustomer (name) VALUES ('Sally');
INSERT INTO widgetCustomer (name) VALUES ('Fred');
SELECT * FROM widgetCustomer;

CREATE TRIGGER stampSale AFTER INSERT ON widgetSale
    BEGIN
        UPDATE widgetSale SET stamp = DATETIME('now') WHERE id = NEW.id;
        UPDATE widgetCustomer SET last_order_id = NEW.id, stamp = DATETIME('now')
            WHERE widgetCustomer.id = NEW.customer_id;
        INSERT INTO widgetLog (stamp, event, username, tablename, table_id)
            VALUES (DATETIME('now'), 'INSERT', 'TRIGGER', 'widgetSale', NEW.id);
    END
;

INSERT INTO widgetSale (item_id, customer_id, quan, price) VALUES (1, 3, 5, 1995);
INSERT INTO widgetSale (item_id, customer_id, quan, price) VALUES (2, 2, 3, 1495);
INSERT INTO widgetSale (item_id, customer_id, quan, price) VALUES (3, 1, 1, 2995);

SELECT * FROM widgetSale;
SELECT * FROM widgetCustomer;
SELECT * FROM widgetLog;

-- restore database
DROP TRIGGER IF EXISTS newWidgetSale;
DROP TRIGGER IF EXISTS updateWidgetSale;
DROP TRIGGER IF EXISTS stampSale;

DROP TABLE IF EXISTS widgetCustomer;
DROP TABLE IF EXISTS widgetSale;
DROP TABLE IF EXISTS widgetLog;



In [4]:

q1 = "CREATE TABLE widgetCustomer ( id INTEGER PRIMARY KEY, name TEXT, last_order_id INT );"
q2 = "CREATE TABLE widgetSale ( id INTEGER PRIMARY KEY, item_id INT, customer_id INT, quan INT, price INT );"

q3 = "INSERT INTO widgetCustomer (name) VALUES ('Bob');"
q4 = "INSERT INTO widgetCustomer (name) VALUES ('Sally');"
q5 = "INSERT INTO widgetCustomer (name) VALUES ('Fred');"

sh.perform_insert_and_stats(q1, 'test.db')
sh.perform_insert_and_stats(q2, 'test.db')
sh.perform_insert_and_stats(q3, 'test.db')
sh.perform_insert_and_stats(q4, 'test.db')
sh.perform_insert_and_stats(q5, 'test.db')

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| CREATE TABLE widgetCustomer ( id INTEGER PRIMARY KEY, name TEXT, last_order_id INT ); |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| CREATE TABLE widgetSale ( id INTEGER PRIMARY KEY, item_id INT, customer_id INT, quan INT, price INT ); |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| INSERT INTO widgetCustomer (name) VALUES ('Bob'); |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| INSERT INTO widgetCustomer (name) VALUES ('Sally'); |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| INSERT INTO widgetCustomer (name) VALUES ('Fred'); |||


In [6]:


q = "SELECT * FROM widgetCustomer;"
sh.query_result_stats(q, "test.db")

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| SELECT * FROM widgetCustomer; |||

There are 3 rows in the query results table
There are 3 columns in the query results table

First 5 rows of the query results:
   id   name last_order_id
0   1    Bob          None
1   2  Sally          None
2   3   Fred          None


In [7]:
# create a trigger

q = """
CREATE TRIGGER newWidgetSale AFTER INSERT ON widgetSale
    BEGIN
        UPDATE widgetCustomer SET last_order_id = NEW.id WHERE widgetCustomer.id = NEW.customer_id;
    END
;
"""
sh.perform_insert_and_stats(q, 'test.db')

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| 
CREATE TRIGGER newWidgetSale AFTER INSERT ON widgetSale
    BEGIN
        UPDATE widgetCustomer SET last_order_id = NEW.id WHERE widgetCustomer.id = NEW.customer_id;
    END
;
 |||


In [8]:
# insert some data to activate the trigger

q1 = "INSERT INTO widgetSale (item_id, customer_id, quan, price) VALUES (1, 3, 5, 1995);"
q2 = "INSERT INTO widgetSale (item_id, customer_id, quan, price) VALUES (2, 2, 3, 1495);"
q3 = "INSERT INTO widgetSale (item_id, customer_id, quan, price) VALUES (3, 1, 1, 2995);"

sh.perform_insert_and_stats(q1, 'test.db')
sh.perform_insert_and_stats(q2, 'test.db')
sh.perform_insert_and_stats(q3, 'test.db')

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| INSERT INTO widgetSale (item_id, customer_id, quan, price) VALUES (1, 3, 5, 1995); |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| INSERT INTO widgetSale (item_id, customer_id, quan, price) VALUES (2, 2, 3, 1495); |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| INSERT INTO widgetSale (item_id, customer_id, quan, price) VALUES (3, 1, 1, 2995); |||


In [9]:
# see what changed
# widget customer was updated by the trigger when widget sale was updated.

q1 = "SELECT * FROM widgetSale;"
q2 = "SELECT * FROM widgetCustomer;"

sh.query_result_stats(q1, "test.db")
sh.query_result_stats(q2, "test.db")

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| SELECT * FROM widgetSale; |||

There are 3 rows in the query results table
There are 5 columns in the query results table

First 5 rows of the query results:
   id  item_id  customer_id  quan  price
0   1        1            3     5   1995
1   2        2            2     3   1495
2   3        3            1     1   2995
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| SELECT * FROM widgetCustomer; |||

There are 3 rows in the query results table
There are 3 columns in the query results table

First 5 rows of the query results:
   id   name  last_order_id
0   1    Bob              3
1   2  Sally              2
2   3   Fred              1


# Preventing updates

In [11]:
# create widget sale table again

q1 = "DROP TABLE IF EXISTS widgetSale;"
q2 = """
CREATE TABLE widgetSale ( id integer primary key, item_id INT, customer_id INTEGER, quan INT, price INT,
    reconciled INT );
"""

sh.perform_insert_and_stats(q1, 'test.db')
sh.perform_insert_and_stats(q2, 'test.db')

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| DROP TABLE IF EXISTS widgetSale; |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| 
CREATE TABLE widgetSale ( id integer primary key, item_id INT, customer_id INTEGER, quan INT, price INT,
    reconciled INT );
 |||


In [12]:
# add some data

q1 = "INSERT INTO widgetSale (item_id, customer_id, quan, price, reconciled) VALUES (1, 3, 5, 1995, 0);"
q2 = "INSERT INTO widgetSale (item_id, customer_id, quan, price, reconciled) VALUES (2, 2, 3, 1495, 1);"
q3 = "INSERT INTO widgetSale (item_id, customer_id, quan, price, reconciled) VALUES (3, 1, 1, 2995, 0);"

sh.perform_insert_and_stats(q1, 'test.db')
sh.perform_insert_and_stats(q2, 'test.db')
sh.perform_insert_and_stats(q3, 'test.db')


### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| INSERT INTO widgetSale (item_id, customer_id, quan, price, reconciled) VALUES (1, 3, 5, 1995, 0); |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| INSERT INTO widgetSale (item_id, customer_id, quan, price, reconciled) VALUES (2, 2, 3, 1495, 1); |||
### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| INSERT INTO widgetSale (item_id, customer_id, quan, price, reconciled) VALUES (3, 1, 1, 2995, 0); |||


In [13]:
q = "SELECT * FROM widgetSale;"
sh.query_result_stats(q, "test.db")

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| SELECT * FROM widgetSale; |||

There are 3 rows in the query results table
There are 6 columns in the query results table

First 5 rows of the query results:
   id  item_id  customer_id  quan  price  reconciled
0   1        1            3     5   1995           0
1   2        2            2     3   1495           1
2   3        3            1     1   2995           0


In [14]:
# make a trigger to prevent updates
q = """
CREATE TRIGGER updateWidgetSale BEFORE UPDATE ON widgetSale
    BEGIN
        SELECT RAISE(ROLLBACK, 'cannot update table "widgetSale"') FROM widgetSale
            WHERE id = NEW.id AND reconciled = 1;
    END
;
"""
sh.perform_insert_and_stats(q, 'test.db')

### Query Summary ###

DATABASE QUERIED: test.db
QUERY: ||| 
CREATE TRIGGER updateWidgetSale BEFORE UPDATE ON widgetSale
    BEGIN
        SELECT RAISE(ROLLBACK, 'cannot update table "widgetSale"') FROM widgetSale
            WHERE id = NEW.id AND reconciled = 1;
    END
;
 |||
